In [1]:
import h5py
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime as dt

from itertools import repeat
import concurrent.futures

from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder


In [2]:
start = dt.now()
print('Job start at : ', start)

filename = '../data/preprocess/ALL/augmentation/export/data_all_all_graycoprops_lpb_histogram_hue_moment_haralick_histogram_hsv_histogram_lab_pyfeats.h5'

df_features = pd.DataFrame()

hf = h5py.File(filename, 'r')

features = list(hf.keys())

features_lst = []
features.sort()

for i in range((len(features) // 20 + 1)):
    features_lst.append(features[20*i:20*(i+1)])



def convert_to_df(lst):
    df = pd.DataFrame()
    for f in lst:
        df[f] = list(hf[f])

    return df

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(tqdm(executor.map(convert_to_df, features_lst), total=len(features_lst)))

hf.close()

df_features = pd.DataFrame()

for df in results:
    df_features = pd.concat([df_features, df], axis=1)

df =df_features.drop_duplicates()

print('Duplicates rows : ', df.shape, df_features.shape, (df_features.shape[1]-df.shape[1]))

df_features['classes'] = df_features.classes.apply(lambda l: l.decode("utf-8"))
df_features = df_features.copy()
print(df_features.shape)

features = [f for f in df_features.columns if f != 'classes']
df_features[features]=MinMaxScaler().fit_transform(df_features[features].to_numpy()).astype(np.float32)

df_features.to_pickle('../data/preprocess/ALL/augmentation/export/data_all_all_graycoprops_lpb_histogram_hue_moment_haralick_histogram_hsv_histogram_lab_pyfeats.pkl')

print('Job ended at ', dt.now(), '\tIt took : ', dt.now() - start)

  9%|▉         | 100/1109 [01:32<13:47,  1.22it/s]/tmp/ipykernel_7047/1679797792.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_features[f] = list(hf[f])
100%|██████████| 1109/1109 [15:10<00:00,  1.22it/s]


(190954, 1109)


In [2]:
1109 / 23

48.21739130434783